In [3]:
import requests
import pandas as pd
import key 

# I have decided to use pandas and array's to store the data. 
# I have also decided to use a class to store the data and to write the data to csv files.
#The Reason that these decisions were made is because it was easiest to manipulate financial
#data using pandas and arrays. The financial data at the endpoint of this API is in JSON format
#and contain many nested arrays. Pandas allows for easy manipulation of these nested arrays.

class Write_Fincial_Data_to_csv: 
    def __init__(self, Url_Base, Symbol, Api_Key):
        self.Url_Base = Url_Base
        self.Symbol = Symbol
        self.Api_Key = Api_Key
        self.netIncome = self.get_netIncome(Url_Base, Symbol, Api_Key)
        self.stock_price = self.get_stock_data(Url_Base, Symbol, Api_Key)
        
        self.save_data_as_csv('netIncome.csv','stock_data.csv')

        #This is intiating everything importing my parameters given outside of the class


    def get_netIncome(self, Url_Base, Symbol, Api_Key):
        url = f"{Url_Base}function=INCOME_STATEMENT&symbol={Symbol}&apikey={Api_Key}"
        response = requests.get(url)
        data = response.json()
        df = pd.json_normalize(data)    
        df = df['quarterlyReports'][0] #The data is enclosed in a sub array called quarterly reports
        df = pd.DataFrame(df) #creating the dataframe
        df = df.set_index('fiscalDateEnding') #This is the header for the date column
        netIncome = df['netIncome'] #This is the header for the net income column
        return netIncome
    
        #net income is a very intersting metric to look at because it is the final statment 
        #of money a company has made after all expenses.
        
    
    def get_stock_data(self, Url_Base, Symbol, Api_Key):
        url = f"{Url_Base}function=TIME_SERIES_WEEKLY_ADJUSTED&symbol={Symbol}&apikey={Api_Key}"
        response = requests.get(url)
        data = response.json()
        weekly_data_unformated = data['Weekly Adjusted Time Series']
        Stock_price = pd.DataFrame.from_dict(weekly_data_unformated, orient='index')
        Stock_price = Stock_price.rename_axis('date')
        Stock_price.index = pd.to_datetime(Stock_price.index)
        return Stock_price
    
        #This is the stock price data. I have decided to use the weekly adjusted time series

    def save_data_as_csv(self, netIncome, Stock_price): 
        self.netIncome.to_csv(netIncome, index=True) #very importatnt includes index in data as first column
        self.stock_price.to_csv(Stock_price, index=True) 

        #netIncome_filename, crudeOil_filename, natGas_filename, stock_price_filename):

# Example usage
Url_Base = "https://www.alphavantage.co/query?" #This is the base url for the API
Symbol = "XOM"  # Ticker symbol for Exxon Mobil
Api_Key =  f"{key.Api_Key}" #This is the API key that I have stored in a seperate file called "key.py

reader = Write_Fincial_Data_to_csv(Url_Base, Symbol, Api_Key) #usage


In [ ]:
import requests
import pandas as pd

#This data is from the Fred API, through alpha vantage, this offers some drawbacks as I can get
#limited usage of the data per day. This is why decided to put it in a different class, and different
#jupiter notebook. Like above I have decided to use pandas and arrays to store the data.

class WriteCommoditiesDataToCsv:
    def __init__(self, Url_Base, Symbol, Api_Key, interval, commodity_type):
        self.Url_Base = Url_Base
        self.Symbol = Symbol
        self.Api_Key = Api_Key
        self.interval = interval
        self.commodity_type = commodity_type

        # Get commodity price and save to CSV
        df_commodity = self.get_commodities_price()
        self.save_data_as_csv(df_commodity, f"{commodity_type}.csv")

    def get_commodities_price(self):
        url = f"{self.Url_Base}function={self.commodity_type}&interval={self.interval}&apikey={self.Api_Key}"
        response = requests.get(url)
        data = response.json()
        #for some reason the data  on this line is that i had a lot of problems because it wans't pre-index,
        #luckly I was able to fix it with this nifty line below.
        df_commodity_price = pd.DataFrame(data['data'], columns=['date', 'value']) 
        #inplace = true is important because it specfies that we are editing the dataframe in place            
        df_commodity_price.set_index('date', inplace=True) 
        #converting to numeric values                                       
        df_commodity_price['value'] = pd.to_numeric(df_commodity_price['value'], errors='coerce') 
        return df_commodity_price                                                                 
        
    def save_data_as_csv(self, df, filename):
        df.to_csv(filename, index=True)

# Example usage
Url_Base = "https://www.alphavantage.co/query?"
Symbol = "XOM"
Api_Key = f"{key.Api_Key}"
interval = "weekly"
commodity_type = "NATURAL_GAS"

reader = WriteCommoditiesDataToCsv(Url_Base, Symbol, Api_Key, interval, commodity_type)
